In [1]:
import numpy as np
import pandas as pd
import time
import ast
import json
import datetime
import pprint


import warnings 
warnings.filterwarnings('ignore')

import logging
from kiteconnect import KiteConnect

from kiteconnect import KiteConnect, KiteTicker
import time
import threading

logging.basicConfig(level=logging.ERROR)

from kiteconnect import KiteConnect
import os
import pickle

pd.options.display.max_rows = 500
pd.options.display.max_columns = 1000

In [2]:
inst_list = [
             "ICICIBANK","HDFCBANK","YESBANK","AXISBANK",\
             "INFY","TCS",\
             "TATAMOTORS","TVSMOTOR",\
             "BAJFINANCE"
            ]


## Plan of Action

- For each ticker, the opening range is constructed
- Based on opening bullish/bearish chandle, the breakout level is determined. Scope to use retracement levels
- Based on Fibonacci Golden ratio Extension , target is determined


### Parsing University Data

In [3]:
import os

data_dir = "E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/"
dirs =[] 
for p,q,r in os.walk(data_dir):
    if not r:
    
        [dirs.append(os.path.join(p,d)) for d in q]
        

In [4]:
dirs

['E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2021',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data January 2022\\Cash IEOD Gold - 2201',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data February 2022\\Cash IEOD Gold - 2202',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data March 2022\\Cash IEOD Gold - 2203',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data April 2022\\Cash Data April 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data May 2022\\Cash Data May 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash Data June 2022\\Cash Data June 2022',
 'E:/CapstoneWQU/Cash Stocks Data-20251004T081638Z-1-001/Cash Stocks Data/2022\\Cash D

In [5]:
dfs ={}

for data_dir in dirs:
    for file in os.listdir(data_dir):
        if not file.endswith('.csv'):
            continue
        if file[0]!=".":
            symbol = file.removesuffix(".csv") 
            if symbol in inst_list :
#                 print(file)
                df = pd.read_csv(data_dir+"/"+file)
                df.columns = [ x.strip()[1:-1] for x in df.columns.tolist() ]
                df.date = pd.to_datetime(df.date)
                prev_df = dfs.get(symbol)
                if prev_df is not None:
                    df = pd.concat([prev_df,df],axis =0 )
                dfs[symbol]=df

## intervals
- 1 min but can be extended for 5 min 

In [6]:
## Sort by date , time and construct 1 min ohlc intervals
for k,v in dfs.items():
    v = v.sort_values(['date','time'],ascending=True)
    v['ohlc_interval'] = pd.to_datetime(v.time).dt.floor('1min').dt.time
    v['o'] = v.groupby(['date','ohlc_interval'])['open'].transform('first')
    v['h'] = v.groupby(['date','ohlc_interval'])['high'].transform('max')
    v['l'] = v.groupby(['date','ohlc_interval'])['low'].transform('min')
    v['c'] = v.groupby(['date','ohlc_interval'])['close'].transform('last')
    v['v'] = v.groupby(['date','ohlc_interval'])['volume'].transform('sum')
#     v = v.drop_duplicates(['date','ohlc_interval'])
    dfs[k] = v

## Code for marking breakout levels, Fibonacci Target 

 - open,high,low,close represent the prices for that particular min
 - ohlc_interval represents the window granularity this row belongs to
    -- Example could be 1 min or 5 min or 15 min
  - o,h,l,c represents the open, high,low,close for that window granularity
  
  - Columns
   - 'bullish' - Whether opening window is bullish or bearish
   - 'OR' - Opening window range
   - 'ORB_breakout_level'- Breakout defined for bullish/bearish
   - 'ORB_entry_level',
   - 'ORB_stoploss_level',
   - 'ORB_golden_ratio_target1',
   - 'ORB_golden_ratio_target2',
   - 'did_break_out',
   - 'did_retrace',
   - 'did_fib_target_hit'

In [7]:
dfs_orb= {}
for symbol in  inst_list:
    df = dfs.get(symbol)

    df['dt'] = pd.to_datetime(df.date).dt.date

    bullish_df= df.groupby('dt').first().apply(lambda row:row['o']-row['c'] > 0,axis=1).to_frame().reset_index()

    bullish_df = bullish_df.rename({0:'bullish'},axis=1)

    OR_df = df.groupby('dt').first().apply(lambda row:row['h']-row['l'],axis=1).to_frame().reset_index()

    OR_df = OR_df.rename({0:'OR'},axis=1)
    
    
    df = df.merge(bullish_df).merge(OR_df)

    ORB_Target1_df = df.groupby('dt').first().apply(lambda row: row['l'] + 1.618 *  row['OR'] if row['bullish'] else row['h'] - 1.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target1_df = ORB_Target1_df.rename({0:'ORB_golden_ratio_target1'},axis=1)
    
    ORB_Target2_df = df.groupby('dt').first().apply(lambda row: row['l'] + 2.618 *  row['OR'] if row['bullish'] else row['h'] - 2.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_Target2_df = ORB_Target2_df.rename({0:'ORB_golden_ratio_target2'},axis=1)

    ORB_breakout_df = df.groupby('dt').first().apply(lambda row: row['h'] if row['bullish'] else row['l'],axis=1).to_frame().reset_index()

    ORB_breakout_df= ORB_breakout_df.rename({0:'ORB_breakout_level'},axis=1)
    
    ORB_entry_level_df = df.groupby('dt').first().apply(lambda row: row['h'] - 0.618 *  row['OR'] if row['bullish'] else row['l'] + 0.618 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_entry_level_df= ORB_entry_level_df.rename({0:'ORB_entry_level'},axis=1)
    
    ORB_stoploss_level_df = df.groupby('dt').first().apply(lambda row: row['h'] - 1 *  row['OR'] if row['bullish'] else row['l'] + 1 *  row['OR'],axis=1).to_frame().reset_index()

    ORB_stoploss_level_df= ORB_stoploss_level_df.rename({0:'ORB_stoploss_level'},axis=1)

    df = df.merge(ORB_breakout_df).merge(ORB_entry_level_df).merge(ORB_stoploss_level_df).merge(ORB_Target1_df).merge(ORB_Target2_df)    
    
    df['did_break_out'] =df.apply(lambda x:x['high'] > x['ORB_breakout_level'] if x['bullish']  else x['low'] < x['ORB_breakout_level'],axis=1)
    df['did_retrace'] =df.apply(lambda x:(x['low'] < x['ORB_entry_level'] and  x['did_break_out']) if x['bullish']  else (x['high'] > x['ORB_entry_level'] and  x['did_break_out']),axis=1)
    df['did_fib_target_hit'] =df.apply(lambda x:x['high'] >= x['ORB_golden_ratio_target1'] if x['bullish']  else x['low'] <= x['ORB_golden_ratio_target1'],axis=1)
    
    
    dfs_orb[symbol]=df

In [ ]:
# dfs_orb.get(inst_list[0]).head(2)

In [8]:
icici_bank = dfs_orb.get(inst_list[0])

In [9]:
icici_bank.query('did_break_out & did_fib_target_hit').head(5)

,ticker,date,time,open,high,low,close,volume,o/i,ohlc_interval,o,h,l,c,v,dt,bullish,OR,ORB_breakout_level,ORB_entry_level,ORB_stoploss_level,ORB_golden_ratio_target1,ORB_golden_ratio_target2,did_break_out,did_retrace,did_fib_target_hit
392,ICICIBANK,2021-01-04,09:30:00,533.20,533.50,532.90,533.50,76393,0,09:30:00,533.20,533.50,532.90,533.50,76393,2021-01-04,True,1.5,532.55,531.623,531.05,533.477,534.977,True,False,True
393,ICICIBANK,2021-01-04,09:31:00,533.50,533.90,533.40,533.65,73424,0,09:31:00,533.50,533.90,533.40,533.65,73424,2021-01-04,True,1.5,532.55,531.623,531.05,533.477,534.977,True,False,True
394,ICICIBANK,2021-01-04,09:32:00,533.70,533.80,533.25,533.35,72505,0,09:32:00,533.70,533.80,533.25,533.35,72505,2021-01-04,True,1.5,532.55,531.623,531.05,533.477,534.977,True,False,True
395,ICICIBANK,2021-01-04,09:33:00,533.35,533.65,533.05,533.50,49344,0,09:33:00,533.35,533.65,533.05,533.50,49344,2021-01-04,True,1.5,532.55,531.623,531.05,533.477,534.977,True,False,True
396,ICICIBANK,2021-01-04,09:34:00,533.45,533.50,533.00,533.00,39419,0,09:34:00,533.45,533.50,533.00,533.00,39419,2021-01-04,True,1.5,532.55,531.623,531.05,533.477,534.977,True,False,True


In [10]:
for inst in inst_list :
    inst_df = dfs_orb.get(inst)
    print('-- instrument --',inst)
    print('Number of days',inst_df.date.nunique())
    print('Number of breakout days',inst_df.query('did_break_out').date.nunique())
    print('Number of breakout days with fib target hit',inst_df.query('did_break_out & did_fib_target_hit').date.nunique())
    print('Number of breakout days with retracement',inst_df.query('did_retrace & did_break_out').date.nunique())

-- instrument -- ICICIBANK
Number of days 496
Number of breakout days 342
Number of breakout days with fib target hit 246
Number of breakout days with retracement 50
-- instrument -- HDFCBANK
Number of days 496
Number of breakout days 333
Number of breakout days with fib target hit 226
Number of breakout days with retracement 35
-- instrument -- YESBANK
Number of days 496
Number of breakout days 356
Number of breakout days with fib target hit 294
Number of breakout days with retracement 92
-- instrument -- AXISBANK
Number of days 496
Number of breakout days 352
Number of breakout days with fib target hit 264
Number of breakout days with retracement 63
-- instrument -- INFY
Number of days 496
Number of breakout days 357
Number of breakout days with fib target hit 244
Number of breakout days with retracement 52
-- instrument -- TCS
Number of days 496
Number of breakout days 327
Number of breakout days with fib target hit 257
Number of breakout days with retracement 56
-- instrument -- TA